# NLP with Bert for Sentiment Analysis

### Importing the libraries

In [ ]:
!pip3 install ktrain

     |████████████████████████████████| 25.2MB 125kB/s 
     |████████████████████████████████| 421.8MB 28kB/s 
     |████████████████████████████████| 6.7MB 38.0MB/s 
     |████████████████████████████████| 983kB 13.0MB/s 
     |████████████████████████████████| 245kB 40.6MB/s 
     |████████████████████████████████| 778kB 39.0MB/s 
     |████████████████████████████████| 471kB 37.0MB/s 
     |████████████████████████████████| 3.9MB 38.0MB/s 
     |████████████████████████████████| 450kB 41.4MB/s 
     |████████████████████████████████| 3.0MB 37.8MB/s 
     |████████████████████████████████| 890kB 31.8MB/s 
     |████████████████████████████████| 1.1MB 37.1MB/s 
  Created wheel for ktrain: filename=ktrain-0.18.5-cp36-none-any.whl size=25253441 sha256=73ebe18aa44f9691c108c887757ca53465676b0baf946526a845c15c6fd4ab19
  Stored in directory: /root/.cache/pip/wheels/29/33/cd/2a24a3b6344bb291ede07a903985664bf747b3ea50d9c05ce4
  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp36-no

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os.path
import ktrain
from ktrain import text

## Part 1: Data Preprocessing

### Loading the IMDB dataset

In [ ]:
dataset = tf.keras.utils.get_file(fname='aclImdb_v1.tar.gz',
                                  origin='https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz',
                                  extract=True)
Imdb_data = os.path.join(os.path.dirname(dataset),'aclImdb')

84131840/84125825 [==============================] - 8s 0us/step


In [ ]:
print(os.path.dirname(dataset))
print(Imdb_data)

/root/.keras/datasets
/root/.keras/datasets/aclImdb


In [ ]:
!ls /root/.keras/datasets/aclImdb/test

labeledBow.feat  neg  pos  urls_neg.txt  urls_pos.txt



### Creating the training and test sets

In [ ]:
(x_train, y_train),(x_test, y_test),preproc = text.texts_from_folder(datadir=Imdb_data, classes=['pos','neg'],
                                                          ngram_range=1,
                                                          maxlen= 500,val_pct=0.11, 
                                                          preprocess_mode ='bert')

detected encoding: utf-8
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


## Part 2: Building the BERT model

In [ ]:
model = text.text_classifier(name='bert',train_data = (x_train,y_train),
                     preproc = preproc, 
                     metrics=['accuracy'])

Is Multi-Label? False
maxlen is 500
done.


In [ ]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 500)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 500)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 500, 768), ( 23440896    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 500, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

## Part 3: Training the BERT model

In [ ]:
leraner = ktrain.get_learner(model = model, 
                   train_data = (x_train, y_train),
                   val_data = (x_test,y_test),
                   multigpu = True,
                   batch_size=6)

In [ ]:
leraner.lr_find(show_plot=True)

In [ ]:
leraner.fit_onecycle(lr=2e-5,epochs=1)



begin training using onecycle policy with max lr of 2e-05...
Train on 25000 samples, validate on 25000 samples
25000/25000 [==============================] - 3469s 139ms/sample - loss: 0.2467 - accuracy: 0.8981 - val_loss: 0.1633 - val_accuracy: 0.9390


In [ ]:
dataset_2 = tf.keras.utils.get_file(fname='trainingandtestdata.zip',
                                  origin='http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip',
                                  extract=True)
dirn = os.path.join(os.path.dirname(dataset_2),'Twitters')

81371136/81363704 [==============================] - 5s 0us/step


In [ ]:
!mkdir /root/.keras/datasets/Twitters
!ls /root/.keras/datasets/Twitters

In [ ]:
!mkdir /root/.keras/datasets/Twitters/train
# !mkdir /root/.keras/datasets/Twitters/test

In [ ]:
!ls /root/.keras/datasets/Twitters

test  train


In [ ]:
df = pd.read_csv("/root/.keras/datasets/training.1600000.processed.noemoticon.csv",
                 encoding='latin-1',
                 header=None)

In [ ]:
df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
df.columns = ['Emotion','Id','Dated','Queried','User','Tweet']

In [ ]:
df.head()

,Emotion,Id,Dated,Queried,User,Tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
df.groupby('Emotion').nunique()

,Emotion,Id,Dated,Queried,User,Tweet
Emotion,,,,,,
0,1,800000,593879,1,415671,790185
4,1,800000,286578,1,376569,793506


In [ ]:
!mkdir /root/.keras/datasets/Twitters/train/mean
!mkdir /root/.keras/datasets/Twitters/train/pos
!touch /root/.keras/datasets/Twitters/train/pos/hello.txt
!touch /root/.keras/datasets/Twitters/train/mean/hello.txt

In [ ]:
# !rm -rf /root/.keras/datasets/Twitters/train

In [ ]:
for i,j in zip(df.loc[:,['Emotion','Tweet']].values,range(18000)):
    if i[0] == 0:
      np.savetxt(r'/root/.keras/datasets/Twitters/train/mean/'+str(j)+"th_mean.txt",list(i[1]),fmt="%s")
    else:
      np.savetxt(r'/root/.keras/datasets/Twitters/train/pos/'+str(j)+"th_pos.txt",list(i[1]),fmt="%s")

In [ ]:
!tail /root/.keras/datasets/Twitters/train/pos/

In [ ]:
df_test = pd.read_csv("/root/.keras/datasets/testdata.manual.2009.06.14.csv",
                      encoding='latin-1',
                      header=None)

In [ ]:
df_test.shape

(498, 6)

In [ ]:
df_test.columns = ['Emotion','Id','Dated','Queried','User','Tweet']

In [ ]:
!rm -f /root/.keras/datasets/Twitters/train/mean/hello.txt
!rm -f /root/.keras/datasets/Twitters/train/pos/hello.txt
# !mkdir /root/.keras/datasets/Twitters/test/mean
# !mkdir /root/.keras/datasets/Twitters/test/pos
# !touch /root/.keras/datasets/Twitters/test/pos/hello.txt
# !touch /root/.keras/datasets/Twitters/test/mean/hello.txt

In [ ]:
for i,j in zip(df.loc[:,['Emotion','Tweet']].values,range(498)):
    if i[0] == 0:
      np.savetxt(r'/root/.keras/datasets/Twitters/test/mean/'+str(j)+"th_mean.txt",
                 list(i[1]),
                 fmt="%s")
    else:
      np.savetxt(r'/root/.keras/datasets/Twitters/test/pos/'+str(j)+"th_pos.txt",
                 list(i[1]),
                 fmt="%s")

In [ ]:
!rm -f /root/.keras/datasets/Twitters/test/mean/hello.txt
!rm -f /root/.keras/datasets/Twitters/test/pos/hello.txt

In [ ]:
new_data_dir = '/root/.keras/datasets/Twitters'

In [ ]:
(x_train, y_train),(x_test, y_test),preproc = text.texts_from_folder(datadir=new_data_dir, classes=['pos','mean'],
                                                          ngram_range=2,
                                                          maxlen= 500,val_pct=0.11, 
                                                          preprocess_mode ='bert',)

detected encoding: utf-8
preprocessing train...
language: cy


Is Multi-Label? False
preprocessing test...
language: cy


In [ ]:
y_train= tf.compat.v1.keras.utils.to_categorical(y_train)
y_test = tf.compat.v1.keras.utils.to_categorical(y_test)

In [ ]:
model = text.text_classifier(name='bert',train_data = (x_train,y_train),
                     preproc = preproc, 
                     metrics=['accuracy'],)

Is Multi-Label? False
maxlen is 500
done.


In [ ]:
learner2 = ktrain.get_learner(model = model, 
                   train_data = (x_train, y_train),
                   val_data = (x_test,y_test),
                   multigpu = True,
                   batch_size=6)

In [ ]:
# learner2.lr_find(start_lr=2e-3,show_plot=True,max_epochs=4)

In [ ]:
learner2.fit_onecycle(lr=2e-3,epochs=1) 



begin training using onecycle policy with max lr of 0.002...
Train on 18000 samples, validate on 498 samples
18000/18000 [==============================] - 6690s 372ms/sample - loss: 7.3328e-07 - accuracy: 1.0000 - val_loss: 2.3938e-10 - val_accuracy: 1.0000


In [ ]:
!pip install tensorflow

In [ ]:
tf.__version__

'2.1.0'